# Real vs Synthetic Images

In this notebook we compare synthetic and ground truth images for the 4 channels in a quiet day and in correspondence of a flare

In [139]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from typing import Dict
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from torch.utils.data import DataLoader
import torch

from sunpy.visualization.colormaps import cm

from sdo.datasets.sdo_dataset import SDO_Dataset
from sdo.datasets.virtual_telescope_sdo_dataset import VirtualTelescopeSDO_Dataset
from sdo.models.vt_models.vt_unet import VT_UnetGenerator

In [150]:
def compute_predictions(timestamp :str , ch_dict, model_key) -> Dict:
    datetime = [[timestamp, timestamp]]
    results_dict = {}
    for ch in ["211", "193", "171", "094"]:
        print(f"-----{ch}------")
        # Load input data - we reload the data each time for convenience, the first 3 channels are
        # assumed to be the inputs and packed in a single tensor
        data = VirtualTelescopeSDO_Dataset(data_basedir='/fdl_sdo_data/SDOMLmm/fdl_sw/SDOMLmm', 
            data_inventory='/home/Valentina/inventory_1904.pkl', 
            instr=["AIA", "AIA", "AIA", "AIA"],
            num_channels = 4,
            channels=ch_dicts[ch]["channels"],
            datetime_range = datetime_range,
            resolution=512,
            )

        if len(data) < 1:
            print("Date not found")

        # Load model - each channel combination has a model with different weights
        model_path = results_path + ch_dicts[ch][model_key]
        model = VT_UnetGenerator(input_shape=[3, 512, 512])
        state_dict = torch.load(model_path)
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k.replace("module.", "") 
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict, strict=False)
        model.eval();

        # Compute prediction
        input_chs = data[0][0].unsqueeze(0)
        output = model(input_chs).detach().numpy().reshape(512, 512)
        gt_img = data[0][1].detach().numpy().reshape(512, 512)

        results_dict[ch] =  {"gt": gt_img, "pred": output, "input": input_chs}
    return results_dict

In [145]:
def plot_4ch_grid(results_dict: Dict, ch_dict: Dict) -> plt.Figure:
    fig, ax = plt.subplots(4, 3, figsize=(15,15))
    i = 0
    for ch in ["211", "193", "171", "094"]:
        gt_img = results_dict[ch]["gt"]
        pred = results_dict[ch]["pred"]
        cmap = ch_dicts[ch]["cmap"]
        if ch in ["211", "171"]:
            vmin1 = vim2 = 0
            vmax1 = vmax2 = 3
            vmin3 = -0.8
            vmax3= 0.8
        elif ch == "193":
            vmin1 = vim2 = 0
            vmax1 = vmax2 = 3
            vmin3 = -0.8
            vmax3= 3
        elif ch == "094":
            vmin1 = vmin2 = 0
            vmax1 = 100
            vmax2 = 0.1
            vmin3 = -0.8
            vmax3= 3
        ax[i, 0].imshow(gt_img, cmap=cmap, origin='lower', vmin=vmin1, vmax= vmax1)
        ax[i, 1].imshow(pred, cmap=cmap, origin='lower', vmin=vmin2, vmax= vmax2)
        ax[i, 2].imshow(pred - gt_img, cmap='seismic', origin='lower', vmin=vmin3, vmax=vmax3)
        i = i + 1
    plt.setp(ax, xticks=[], yticks=[])
    plt.show()
    return fig

In [154]:
results_path = '/fdl_sdo_data/bucket2/EXPERIMENT_RESULTS/VIRTUAL_TELESCOPE/'

ch_dict = {"211": {"model_nr": 'vale_exp_20/0600_vale_exp_20_model.pth',
                    "model_root": 'vale_exp_23/0600_vale_exp_23_model.pth',
                    "channels": ["0094", "0193", "0171", "0211"],
                    "cmap": cm.cmlist.get("sdoaia211"),
                   }
            ,
            "193": {"model_nr": '/vale_exp_13bis/0600_vale_exp_13bis_model.pth',
                    "model_root": 'vale_exp_25/0600_vale_exp_25_model.pth',
                    "channels": ["0211", "0094", "0171", "0193"],
                    "cmap": cm.cmlist.get("sdoaia193"),
                   }
            ,
            "171": {"model_nr": '/vale_exp_14bis/0600_vale_exp_14bis_model.pth',
                    "model_root": 'vale_exp_26/0600_vale_exp_26_model.pth',
                    "channels": ["0094", "0193", "0211", "0171"],
                    "cmap": cm.cmlist.get("sdoaia171")
                   }
            ,
            "094": {"model_nr": '/vale_exp_18/0600_vale_exp_18_model.pth',
                    "model_root": 'vale_exp_27/0400_vale_exp_27_model.pth',
                    "channels": ["0193", "0171", "0211", "0094"],
                    "cmap": cm.cmlist.get("sdoaia094")}
           }

## Images on quiet Sun

In [147]:
quiet_day = '2011-02-10 00:00:00'

### Compute predictions

In [ ]:
nr_results_dict = compute_predictions(quiet_day, ch_dict, "model_nr")

-----211------


/home/Valentina/expanding-sdo-capabilities/src/sdo/datasets/dates_selection.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sel_df['date'] = sel_df.apply(from_row_to_date, axis=1)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6701: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


-----193------


/home/Valentina/expanding-sdo-capabilities/src/sdo/datasets/dates_selection.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sel_df['date'] = sel_df.apply(from_row_to_date, axis=1)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6701: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


-----171------


/home/Valentina/expanding-sdo-capabilities/src/sdo/datasets/dates_selection.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sel_df['date'] = sel_df.apply(from_row_to_date, axis=1)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6701: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
root_results_dict = compute_predictions(quiet_day, ch_dict, "model_root")

### Plot images

In [ ]:
fig = plot_4ch_grid(nr_results_dict, ch_dict)

In [ ]:
fig = plot_4ch_grid(root_results_dict, ch_dict)

## Images on active Sun

In [ ]:
active_day = '2011-02-15 02:00:00'

### Compute predictions

In [ ]:
nr_results_dict_active = compute_predictions(active_day, ch_dict, "model_nr")

In [ ]:
root_results_dict_active = compute_predictions(active_day, ch_dict, "model_root")

### Plot images

In [ ]:
fig = plot_4ch_grid(nr_results_dict_active, ch_dict)

In [ ]:
fig = plot_4ch_grid(root_results_dict_active, ch_dict)